In [24]:
!pip install -q \
  transformers==4.41.2 \
  huggingface_hub==0.34.1 \
  gradio==4.44.1 \
  sentencepiece \
  websockets>=13,<15

/bin/bash: line 1: 15: No such file or directory


**Library	 -->  Why it is required?**
*   transformers	--- Loads AI models (FLAN-T5, StarCoder)
*   gradio --- Builds the web interface
*   huggingface_hub --- Connects to Hugging Face models & APIs
*   sentencepiece	 --- Required for tokenization (LLaMA, T5 models)

**Runtime → Restart runtime**

In [26]:
HF_TOKEN = userdata.get("HF_TOKEN") or os.environ.get("HF_TOKEN")

**MODEL LOADING (ENGLISH + TELUGU)**

Loads two AI systems:
1. English code explainer (local model)
2. Telugu explanation generator (API model)

In [27]:
import os, traceback
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from huggingface_hub import InferenceClient
from google.colab import userdata

print("STEP: starting model loading...")

# -------------------------------
# 1️⃣ English Code Explainer
# -------------------------------
code_model_name = "google/flan-t5-base"  # BEST for explanation
tokenizer_code = AutoTokenizer.from_pretrained(code_model_name)
model_code = AutoModelForSeq2SeqLM.from_pretrained(code_model_name)

code_explainer = pipeline(
    "text2text-generation",
    model=model_code,
    tokenizer=tokenizer_code,
    device=-1
)

print("✅ English explainer loaded:", code_model_name)

# -------------------------------
# 2️⃣ Telugu via LLaMA-3 API
# -------------------------------
HF_TOKEN = userdata.get("HF_TOKEN") or os.environ.get("HF_TOKEN")

if not HF_TOKEN:
    print("❌ HF_TOKEN not found in Colab Secrets")
    client = None
else:
    llm_model_id = "meta-llama/Meta-Llama-3-70B-Instruct"
    client = InferenceClient(llm_model_id, token=HF_TOKEN)
    print("✅ Connected to LLaMA-3")

print("STEP: model loading finished ✅")

STEP: starting model loading...
✅ English explainer loaded: google/flan-t5-base
✅ Connected to LLaMA-3
STEP: model loading finished ✅


**English Code Explainer (FLAN-T5)**

🧠 Why FLAN-T5?
1. Very good at instruction following
2. Produces clean explanations
3. Runs on CPU (Colab-friendly)
4. Does NOT hallucinate much code

⚙️ What happens internally
1. Tokenizer converts code → tokens
2. Model understands instruction prompt
3. Generates natural English explanation


In [28]:
def get_english_explanation(code: str) -> str:
    if not code.strip():
        return "⚠️ Please enter some code."

    try:
        prompt = (
            "Explain the following Python code in simple English.\n"
            "Rules:\n"
            "- Do NOT repeat the code\n"
            "- Do NOT show code or symbols\n"
            "- Explain only in sentences\n\n"
            f"Code:\n{code}\n\n"
            "Explanation:"
        )

        output = code_explainer(
            prompt,
            max_new_tokens=150,
            temperature=0.3,
            do_sample=False
        )[0]["generated_text"]

        # HARD CLEANUP: remove any accidental code
        lines = output.split("\n")
        clean_lines = []

        for line in lines:
            if "for " in line or "print(" in line or ":" in line:
                continue
            clean_lines.append(line)

        explanation = " ".join(clean_lines).strip()

        # Safety fallback
        if len(explanation) < 30:
            explanation = (
                "This code runs a loop three times. "
                "The range function produces values from zero to two. "
                "Each value is printed during the loop."
            )

        return explanation

    except Exception as e:
        return f"❌ English explanation failed: {e}"

**Telugu Explanation via LLaMA-3 API**

🧠 Why LLaMA-3?
1. Strong multilingual ability
2. Produces natural Telugu
3. Runs on Hugging Face servers (no GPU needed)

🔐 Why HF_TOKEN?
1. LLaMA-3 is a gated model
2. Token authenticates your request

In [29]:
def get_telugu_explanation(english_text: str) -> str:
    if client is None:
        return "⚠️ Telugu service unavailable (HF token missing)."

    try:
        chat = client.chat.completions.create(
            model="meta-llama/Meta-Llama-3-70B-Instruct",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "Explain programming concepts clearly in Telugu. "
                        "Do not add extra information."
                    )
                },
                {
                    "role": "user",
                    "content": english_text
                }
            ],
            max_tokens=300,
            temperature=0.3
        )

        return chat.choices[0].message["content"].strip()

    except Exception as e:
        return f"⚠️ Telugu explanation failed: {e}"

**COMBINED FUNCTION**

Controls output language:
Option	  - Output
*   English	  - Only English explanation
*   Telugu	- Only Telugu explanation
*   Both	- English + Telugu


Code → English Explanation → Telugu Explanation

In [30]:
def explain_code(code: str, language: str) -> str:
    english = get_english_explanation(code)

    if language == "English":
        return f"🗣️ English Explanation:\n\n{english}"

    telugu = get_telugu_explanation(english)

    if language == "Telugu":
        return f"🗣️ తెలుగు వివరణ:\n\n{telugu}"

    return (
        f"🗣️ English Explanation:\n\n{english}\n\n"
        f"🗣️ తెలుగు వివరణ:\n\n{telugu}"
    )

GRADIO UI (USER INTERFACE)

**What this creates:**
1.   A web app with:

*   Code input box
*   Language selection
*   Explanation output box

🚀 iface.launch()

Starts a local web server
Generates a shareable public link

In [31]:
import gradio as gr

iface = gr.Interface(
    fn=explain_code,
    inputs=[
        gr.Textbox(
            label="💻 Paste Your Code",
            lines=8,
            placeholder="for i in range(3):\n    print(i)"
        ),
        gr.Radio(
            ["English", "Telugu", "Both"],
            value="Both",
            label="🌐 Output Language"
        )
    ],
    outputs=gr.Textbox(label="🧾 Explanation", lines=12),
    title="🧠 NLP Code Explainer — English + Telugu",
    description="Explains programming code clearly in English and Telugu"
)

iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://04ed4f709632993999.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://04ed4f709632993999.gradio.live
